In [1]:
import pandas as pd
import requests
import json
from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key=' ')

# /v2/top-headlines/
sources = newsapi.get_sources()


In [2]:
news_station_ids = "the-verge,the-wall-street-journal,the-washington-post,the-washington-times,time,usa-today,vice-news,wired,the-next-web,techcrunch,techcrunch-cn,techradar,the-american-conservative,reddit-r-all,reuters,national-geographic,national-review,nbc-news,news24,new-scientist,news-com-au,newsweek,new-york-magazine,next-big-future,independent,google-news,google-news-ar,google-news-au,google-news-br,google-news-ca,google-news-fr,google-news-in,google-news-is,google-news-it,google-news-ru,google-news-sa,google-news-uk,bild,blasting-news-br,bleacher-report,bloomberg,breitbart-news,business-insider,business-insider-uk,buzzfeed,cbc-news,cbs-news,cnn,bbc-news,abc-news,abc-news-au,aftenposten,al-jazeera-english"

In [ ]:
for source in sources['sources']:
    print(source['id'])

In [ ]:
for i in range(5,327):
    all_articles = newsapi.get_everything(sources=news_station_ids,
                                    #   domains='bbc.co.uk,techcrunch.com',
                                    from_param='2024-02-01',
                                      #  to='2024-02-01',
                                      language='en',
                                      # sort_by='relevancy',
                                      page=i)
    with open(f'../data/file_{i}.json', 'w') as json_file:
        json.dump(all_articles, json_file)
    

In [2]:
import os

directory = '../data/'
dataframes = []

for filename in os.listdir(directory):
    if filename.endswith('.json'):  
        file_path = os.path.join(directory, filename)  
        with open(file_path, 'r') as f:  
            data = json.load(f) 
            df = pd.json_normalize(data, record_path=['articles'])
            dataframes.append(df)

full_df = pd.concat(dataframes, ignore_index=True)

In [19]:
full_df.head(1)

,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,https://www.facebook.com/bbcnews,Students unable to afford food for days - union,A student union leader says the situation for ...,https://www.bbc.co.uk/news/uk-england-lancashi...,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-02-17T09:25:15Z,Students caught up in the cost-of-living crisi...,bbc-news,BBC News


In [20]:
full_df.shape[0]

500

In [23]:
full_df['title'].iloc[0]

'Students unable to afford food for days - union'

In [26]:
import gensim
import gensim.corpora as corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords
import nltk
import pyLDAvis.gensim_models
import pyLDAvis
import re
from nltk.stem.porter import PorterStemmer

# Assuming you already have your DataFrame loaded as full_df
# full_df = pd.read_csv('your_data.csv') # Example if you're loading from a CSV

# 1. Pre-process the text data
nltk.download('stopwords')
stop_words = stopwords.words('english')
porter = PorterStemmer()

def preprocess(text):
    text = re.sub(r'\W', ' ', str(text))
    text = text.lower()
    text = text.split()
    text = [porter.stem(word) for word in text if word not in stop_words]
    text = [word for word in text if len(word) > 3] # removing short words
    return text

# Apply preprocessing to each title
data = full_df['title'].map(preprocess)

# 2. Create Dictionary and Corpus
id2word = corpora.Dictionary(data)
corpus = [id2word.doc2bow(text) for text in data]

# 3. Build LDA model
lda_model = LdaModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=10, 
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto')

# 4. Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(vis, 'lda_visualization.html')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jakub\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [3]:
import spacy
from transformers import pipeline

# Load Spacy's English model
nlp = spacy.load("en_core_web_sm")

# Load sentiment analysis pipeline from Hugging Face
sentiment_pipeline = pipeline("sentiment-analysis")

# Example text
texts = ["The battery life of this smartphone is too short", "I love the camera on this phone, but the interface is clunky."]

# Function to perform ABSA
def absa(texts):
    results = []
    for text in texts:
        doc = nlp(text)
        for token in doc:
            # Assuming nouns as aspects for simplicity
            if token.pos_ == "NOUN":
                aspect = token.text
                # Get sentiment for the aspect
                sentiment = sentiment_pipeline(text)
                results.append({"aspect": aspect, "sentiment": sentiment[0]})
    return results

# Perform ABSA on the example texts
absa_results = absa(texts)
for result in absa_results:
    print(f"Aspect: {result['aspect']}, Sentiment: {result['sentiment']['label']}, Score: {result['sentiment']['score']}")



c:\Users\Jakub\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
model.safetensors: 100%|██████████| 268M/268M [00:16<00:00, 16.7MB/s] 
c:\Users\Jakub\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jakub\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To supp

Aspect: battery, Sentiment: NEGATIVE, Score: 0.999439537525177
Aspect: life, Sentiment: NEGATIVE, Score: 0.999439537525177
Aspect: smartphone, Sentiment: NEGATIVE, Score: 0.999439537525177
Aspect: camera, Sentiment: NEGATIVE, Score: 0.9987188577651978
Aspect: phone, Sentiment: NEGATIVE, Score: 0.9987188577651978
Aspect: interface, Sentiment: NEGATIVE, Score: 0.9987188577651978


In [4]:
absa_results

[{'aspect': 'battery',
  'sentiment': {'label': 'NEGATIVE', 'score': 0.999439537525177}},
 {'aspect': 'life',
  'sentiment': {'label': 'NEGATIVE', 'score': 0.999439537525177}},
 {'aspect': 'smartphone',
  'sentiment': {'label': 'NEGATIVE', 'score': 0.999439537525177}},
 {'aspect': 'camera',
  'sentiment': {'label': 'NEGATIVE', 'score': 0.9987188577651978}},
 {'aspect': 'phone',
  'sentiment': {'label': 'NEGATIVE', 'score': 0.9987188577651978}},
 {'aspect': 'interface',
  'sentiment': {'label': 'NEGATIVE', 'score': 0.9987188577651978}}]